In [3]:
import pandas as pd
import plotly.express as px
import networkx as nx
import matplotlib.pyplot as plt
import chart_studio
import chart_studio.plotly as py
from bokeh.io import output_notebook, show, save
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8, Accent
from bokeh.io import output_notebook, show, save
from bokeh.models import Scatter, Range1d, Circle, ColumnDataSource, MultiLine, LabelSet, EdgesAndLinkedNodes, NodesAndLinkedEdges
from bokeh.plotting import figure, from_networkx
output_notebook()

Loading BokehJS ...

In [4]:
df= pd.read_csv('tree_data.csv')
df = df.sort_values(by='Name')
fig = px.treemap(
    names = df['Name'],
    parents = df['Parent']
)
fig.update_traces(root_color="darkgray")
fig.update_layout(margin = dict(t=100, l=50, r=50, b=50))
fig.show()

In [5]:

username = 'your_username'
api_key = 'your_api_key'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
py.plot(fig, filename = 'english tree demo', auto_open=False)

In [7]:
palette = plt.cm.get_cmap('Paired').colors
groups = pd.read_csv('groups2.csv')
B = nx.Graph()
faculty = {k: {'label':k, 'bipartite':1, 'color': Accent[3][0]} for k in groups['English']}
faculty = [(k, faculty[k]) for k in faculty]
group_nodes = [(group, {'label':group, 'bipartite':0, 'color': Accent[3][1]}) for group in groups.columns]
#B.add_nodes_from(groups['English'], labels={k:k for k in groups['English']}, bipartite=0)
B.add_nodes_from(faculty)
B.add_nodes_from(group_nodes)
for column in groups.columns:
    for value in groups[column].dropna():
        B.add_edge(column, value)
degrees = dict(nx.degree(B))
for key in degrees:    
    degrees[key] *=  2
nx.set_node_attributes(B, name='degree', values=degrees)
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(B)])
nx.set_node_attributes(B, name='adjusted_node_size', values=adjusted_node_size)

In [8]:
#Choose a title!
title = 'English Department Research Foci'

#pos = nx.nx_agraph.graphviz_layout(B, prog="twopi", root=0)
pos= nx.spring_layout(B)
#pos= nx.circular_layout(B)

node_highlight_color = Accent[7][5]
edge_highlight_color = Accent[7][5]

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Label", "@index"),  ]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-.3, .3), y_range=Range1d(-.3,.3), title=title, plot_width=1000)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
#network_graph = from_networkx(B, nx.multipartite_layout(B, subset_key='bipartite'), scale=10, center=(0, 0))
network_graph = from_networkx(B, pos, scale=10, center=(0, 0))


#Set node size and color
network_graph.node_renderer.glyph = Circle(size='adjusted_node_size', fill_color='color')

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_color='white', line_alpha=0.5, line_width=1)
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=4)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=4)


#Add network graph to the plot
plot.renderers.append(network_graph)

    #Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
#network_graph.selection_policy = EdgesAndLinkedNodes()
network_graph.inspection_policy = NodesAndLinkedEdges()

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(B.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
save(plot, filename=f"{title}.html")

/Users/user/opt/anaconda3/lib/python3.7/site-packages/bokeh/io/saving.py:142: UserWarning:

save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN

/Users/user/opt/anaconda3/lib/python3.7/site-packages/bokeh/io/saving.py:154: UserWarning:

save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'



'/Users/user/Library/CloudStorage/Box-Box/Teaching (CMU)/Admin/English Research Tree/English Department Research Foci.html'